In [1]:
!pip install wandb boto3 pyarrow

In [2]:
# Configure some constants for the demo

PROJECT_NAME = 'diabetes-vaccines-notebook'
NOTEBOOK_NAME = 'train-data.ipynb'
ARTIFACT_NAME = 'training-data'
RESOURCES = ['Patient', 'Immunization', 'Condition']
START_BIRTH_DATE = '1920-01-01'
END_BIRTH_DATE = '2020-01-01'
IMMUNIZATION_CODE = '08' # Hepatitis B
CONDITION_CODE = '73211009' # Diabetes

# Configure notebook and login to wandb
configure_notebook_session(NOTEBOOK_NAME)
wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find Untitled.ipynb.
wandb: Currently logged in as: louism. Use `wandb login --relogin` to force relogin


In [ ]:
# Start a new wandb run to track this script
run = wandb.init(
  project=PROJECT_NAME,
  job_type="train_model",
  config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
  }
)

In [3]:
import random
import pandas as pd

ARTEFACT_FULL_NAME = f'louism/{PROJECT_NAME}/{ARTIFACT_NAME}:v0'
artifact = run.use_artifact(ARTEFACT_FULL_NAME, type='dataset')
artifact_dir = artifact.download()
df = pd.read_parquet(artifact_dir, engine='pyarrow')

print(df.head())

wandb:   1 of 1 files downloaded.  


   is_vaccinated has_diabetes  count(id)
0           True        False         18
1           True         None          2
2          False        False        114
3          False         True          8
4          False         None          1


acc,▁▅▆▅▅▇██
loss,█▄▆▄▃▂▁▂
acc,0.82467
loss,0.22049


In [ ]:
# Simulate training
epochs = 100
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    
    # log metrics to wandb
    run.log({"acc": acc, "loss": loss})

In [ ]:
# Save model as a Wandb artifact
file = './model.pt' # model.save("path/to/model.pt")
open(file, 'a').close()

art = wandb.Artifact(f'{PROJECT_NAME}-{wandb.run.id}', type="model")
art.add_file("./model.pt", "model.pt")
run.log_artifact(art)

In [ ]:
# Finish tracking
run.finish()